In [43]:
# import pyspark to process large files
from pyspark.sql import SparkSession
from pyspark.sql.functions import col, asc, desc, to_timestamp

In [44]:
# create a new spark session
spark = SparkSession\
    .builder\
    .master('local[*]')\
    .config("spark.driver.memory", "4g")\
    .appName('process_tripdata')\
    .getOrCreate()
print("Spark version: ", spark.version)

Spark version:  2.4.0


In [45]:
# read 2019-01 data
jan_data = spark\
    .read\
    .csv('/media/felipe/Files/repos/tcc/nyc_data/csv/2019/01/fhv_tripdata_2019-01.csv', header=True)

In [6]:
# show jan_data head
jan_data.printSchema()
jan_data.show(5)

root
 |-- dispatching_base_num: string (nullable = true)
 |-- pickup_datetime: string (nullable = true)
 |-- dropoff_datetime: string (nullable = true)
 |-- PULocationID: string (nullable = true)
 |-- DOLocationID: string (nullable = true)
 |-- SR_Flag: string (nullable = true)

+--------------------+-------------------+-------------------+------------+------------+-------+
|dispatching_base_num|    pickup_datetime|   dropoff_datetime|PULocationID|DOLocationID|SR_Flag|
+--------------------+-------------------+-------------------+------------+------------+-------+
|              B00001|2019-01-01 00:30:00|2019-01-01 02:51:55|        null|        null|   null|
|              B00001|2019-01-01 00:45:00|2019-01-01 00:54:49|        null|        null|   null|
|              B00001|2019-01-01 00:15:00|2019-01-01 00:54:52|        null|        null|   null|
|              B00008|2019-01-01 00:19:00|2019-01-01 00:39:00|        null|        null|   null|
|              B00008|2019-01-01 00:27:00

In [7]:
# get jan data count -> 23,130,810
# print(jan_data.count())

23130810


In [46]:
# filter off null PU and DO location IDs
jan_data.createOrReplaceTempView("JAN_DATA_TEMP_VIEW")
jan_data = spark.sql("SELECT * FROM JAN_DATA_TEMP_VIEW WHERE PULocationID IS NOT NULL AND DOLocationID IS NOT NULL")
jan_data.show(5)

+--------------------+-------------------+-------------------+------------+------------+-------+
|dispatching_base_num|    pickup_datetime|   dropoff_datetime|PULocationID|DOLocationID|SR_Flag|
+--------------------+-------------------+-------------------+------------+------------+-------+
|              B00254|2019-01-01 00:33:03|2019-01-01 01:37:24|         140|          52|   null|
|              B00254|2019-01-01 00:03:00|2019-01-01 00:34:25|         141|         237|   null|
|              B00254|2019-01-01 00:45:48|2019-01-01 01:26:01|         237|         236|   null|
|              B00254|2019-01-01 00:37:39|2019-01-01 01:44:59|         162|          85|   null|
|              B00254|2019-01-01 00:35:06|2019-01-01 01:30:21|         237|         246|   null|
+--------------------+-------------------+-------------------+------------+------------+-------+
only showing top 5 rows



In [9]:
# get jan data count after filtering -> 21,306,221
# print(jan_data.count())

21306221


In [10]:
# convert PU and DO location ID cols to int
jan_data = jan_data.withColumn("PULocationID", jan_data.PULocationID.cast('int'))
jan_data = jan_data.withColumn("DOLocationID", jan_data.DOLocationID.cast('int'))

In [11]:
# view jan_data schema after casting PU and DO cols to int
jan_data

DataFrame[dispatching_base_num: string, pickup_datetime: string, dropoff_datetime: string, PULocationID: int, DOLocationID: int, SR_Flag: string]

In [12]:
# load taxi zones
taxi_zones = spark.read.csv('/media/felipe/Files/repos/tcc/nyc_data/csv/taxi_zone_lookup.csv', header=True)

In [37]:
# show taxi_zones head
taxi_zones.show(5)

+----------+-------------+--------------------+------------+
|LocationID|      Borough|                Zone|service_zone|
+----------+-------------+--------------------+------------+
|         1|          EWR|      Newark Airport|         EWR|
|         2|       Queens|         Jamaica Bay|   Boro Zone|
|         3|        Bronx|Allerton/Pelham G...|   Boro Zone|
|         4|    Manhattan|       Alphabet City| Yellow Zone|
|         5|Staten Island|       Arden Heights|   Boro Zone|
+----------+-------------+--------------------+------------+
only showing top 5 rows



In [47]:
# get how many trips departured from or went to unknown zones
jan_data = jan_data.where("PULocationID < 264 AND DOLocationID < 264")

In [15]:
# count how many trips did not depart or arrive to unknown zones -> 19,847,478
# print(jan_data.count())

19847478


In [48]:
# remove rows where PU and DO cols are 0
jan_data.createOrReplaceTempView("JAN_DATA_TEMP_VIEW")
jan_data = spark.sql("SELECT * FROM JAN_DATA_TEMP_VIEW WHERE PULocationID <> 0 AND DOLocationID <> 0")

In [17]:
# count how many trips PU and DO locations are 0 -> 19,847,263
# print(jan_data.count())

19847263


In [18]:
# get greatest PU and DO location ID to begin assembly of OD dataframe
# jan_data.createOrReplaceTempView("JAN_DATA_TEMP_VIEW")
# spark.sql("SELECT MAX(PULocationID) AS MAX_PU, MAX(DOLocationID) AS MAX_DO FROM JAN_DATA_TEMP_VIEW")\
#     .first()

Row(MAX_PU=263, MAX_DO=263)

In [19]:
# begin OD matrix dataframe assembly
od_size = 264 # 263 + 1 to account for 0-index in list
od_data = [[0 for x in range(od_size)] for y in range(od_size)]
od_cols = [str(x) for x in range(od_size)]

In [20]:
# fill january OD matrix dataframe
jan_data_collect = jan_data.toLocalIterator()
for row in jan_data_collect:
    origin = row.PULocationID
    destination = row.DOLocationID
    od_data[origin][destination] += 1

In [21]:
# create OD dataframe from OD matrix
od_dataframe = spark.createDataFrame(data=od_data,schema=od_cols)

In [22]:
# create and populate list of max trips from to location
od_dataframe_collect = od_dataframe.toLocalIterator()
origin = 0
od_greatest_values = []

for row in od_dataframe_collect:
    max_val = max(list(row))
    destination = row.index(max_val)
    od_greatest_values.append((origin, destination, max_val))
    origin += 1

# create dataframe from od_greatest_values list
od_gr_df_cols = ["PULocationID", "DOLocationID", "TripQty"]
od_gr_df = spark.createDataFrame(data=od_greatest_values, schema=od_gr_df_cols)

In [23]:
# show od_gr_df head
od_gr_df.show(5)

+------------+------------+-------+
|PULocationID|DOLocationID|TripQty|
+------------+------------+-------+
|           0|           0|      0|
|           1|         231|     87|
|           2|         117|      3|
|           3|          51|   2598|
|           4|          79|   4539|
+------------+------------+-------+
only showing top 5 rows



In [24]:
# order by TripQty to find find the top destinations
od_gr_df = od_gr_df.orderBy(col("TripQty").desc())
od_gr_df.show(10)

+------------+------------+-------+
|PULocationID|DOLocationID|TripQty|
+------------+------------+-------+
|          76|          76|  38431|
|          26|          26|  33860|
|          61|          61|  32658|
|          39|          39|  29426|
|           7|           7|  26702|
|         181|         181|  26328|
|         129|         129|  25298|
|          14|          14|  23169|
|          37|          37|  21124|
|          36|          37|  19902|
+------------+------------+-------+
only showing top 10 rows



In [25]:
# extract january top OD data
jan_top_ods = od_gr_df.join(taxi_zones,od_gr_df.PULocationID == taxi_zones.LocationID, "inner")\
    .drop("Borough", "service_zone")

In [26]:
# save january top ODs to file
try:
    jan_top_ods\
        .repartition(1)\
        .write.format("com.databricks.spark.csv")\
        .option("header", "true")\
        .save("/media/felipe/Files/repos/tcc/nyc_data/csv/2019/01/top_ods.csv")
except:
    print("Could not write top_ods to file or file already exists")

Could not write to file or file already exists


In [27]:
# set up helper lists for trip division by time of day
jan_pickup_times = jan_data.select("pickup_datetime")
jan_pickup_times = jan_pickup_times.withColumn("pickup_datetime",to_timestamp("pickup_datetime"))
jan_pickup_times.printSchema()
jan_pickup_times.show(5)

root
 |-- pickup_datetime: timestamp (nullable = true)

+-------------------+
|    pickup_datetime|
+-------------------+
|2019-01-01 00:33:03|
|2019-01-01 00:03:00|
|2019-01-01 00:45:48|
|2019-01-01 00:37:39|
|2019-01-01 00:35:06|
+-------------------+
only showing top 5 rows



In [29]:
# count how many trips per day period
# morning: 0:00 to 5:59
# day: 6:00 to 11:59
# noon: 12:00 to 17:59
# night: 18:00 to 23:59
# weekday is 0-index starting monday (0), then tuesday (1), etc.

MORNING=1
DAY=2
NOON=3
NIGHT=4
trip_groups_schema = ['day of week', 'morning', 'day', 'noon', 'night']
trip_groups = [['mon', 0, 0, 0, 0],\
               ['tue', 0, 0, 0, 0],\
               ['wed', 0, 0, 0, 0],\
               ['thu', 0, 0, 0, 0],\
               ['fri', 0, 0, 0, 0],\
               ['sat', 0, 0, 0, 0],\
               ['sun', 0, 0, 0, 0]]

jan_pickup_times_collect = jan_pickup_times.toLocalIterator()
rows_with_issues = []

for row in jan_pickup_times_collect:
    trip_weekday = row.pickup_datetime.weekday()
    trip_hour = row.pickup_datetime.hour
    if 0 <= trip_hour <= 5:
        trip_groups[trip_weekday][MORNING] += 1
    elif 6 <= trip_hour <= 11:
        trip_groups[trip_weekday][DAY] += 1
    elif 12 <= trip_hour <= 17:
        trip_groups[trip_weekday][NOON] += 1
    elif 18 <= trip_hour <= 23:
        trip_groups[trip_weekday][NIGHT] += 1

In [33]:
# create DF from trip dow group matrix
trip_dow_df = spark.createDataFrame(data=trip_groups, schema=trip_groups_schema)

In [35]:
# show trip_dow_df
trip_dow_df.show()

+-----------+-------+------+------+-------+
|day of week|morning|   day|  noon|  night|
+-----------+-------+------+------+-------+
|        mon| 199734|660084|725648| 750663|
|        tue| 460169|779835|827451| 942559|
|        wed| 229950|821136|838730|1000248|
|        thu| 249241|916219|934264|1106713|
|        fri| 235848|679735|762618|1054765|
|        sat| 479668|531886|892784|1138060|
|        sun| 571719|464548|816346| 776642|
+-----------+-------+------+------+-------+



In [36]:
# write trip_dow_df to file
try:
    trip_dow_df\
        .repartition(1)\
        .write.format("com.databricks.spark.csv")\
        .option("header", "true")\
        .save("/media/felipe/Files/repos/tcc/nyc_data/csv/2019/01/trip_dow_time.csv")
except:
    print("Could not write trip_dow_time to file or file already exists")

In [49]:
# get trips that were shared
jan_data.createOrReplaceTempView("JAN_DATA_TEMP_VIEW")
shared_trips = spark.sql("SELECT * FROM JAN_DATA_TEMP_VIEW WHERE SR_Flag IS NOT NULL")
shared_trips.show(5)

+--------------------+-------------------+-------------------+------------+------------+-------+
|dispatching_base_num|    pickup_datetime|   dropoff_datetime|PULocationID|DOLocationID|SR_Flag|
+--------------------+-------------------+-------------------+------------+------------+-------+
|              B02395|2019-01-01 00:50:26|2019-01-01 01:16:07|          48|         144|      1|
|              B02395|2019-01-01 00:51:20|2019-01-01 01:16:07|          48|         144|      2|
|              B02395|2019-01-01 00:56:24|2019-01-01 01:10:11|         181|          54|      1|
|              B02395|2019-01-01 00:58:12|2019-01-01 01:02:23|         181|         181|      2|
|              B02395|2019-01-01 00:24:44|2019-01-01 00:29:54|         126|         168|      1|
+--------------------+-------------------+-------------------+------------+------------+-------+
only showing top 5 rows



In [50]:
# count how many trips were shared -> 5,361,332

# As per the documentation, there are trips that were flagged as shareable.
# however, this does not mean that it was, since Lyft flags as shared even though
# the original rider wasnt matched with someone else.
# For the purposes of this study, we will analyze the users intent to share.

print('shared jan trips', shared_trips.count())

shared jan trips 5361332
